index

gender: 성별

car: 차량 소유 여부

reality: 부동산 소유 여부

child_num: 자녀 수

income_total: 연간 소득


income_type: 소득 분류
							['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
edu_type: 교육 수준
							['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']

family_type: 결혼 여부
							['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']

house_type: 생활 방식
							['Municipal apartment', 'House / apartment', 'With parents',

							 'Co-op apartment', 'Rented apartment', 'Office apartment']

DAYS_BIRTH: 출생일


							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미


DAYS_EMPLOYED: 업무 시작일

							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미
							양수 값은 고용되지 않은 상태를 의미함

FLAG_MOBIL: 핸드폰 소유 여부

work_phone: 업무용 전화 소유 여부

phone: 전화 소유 여부

email: 이메일 소유 여부

occyp_type: 직업 유형													

family_size: 가족 규모

begin_month: 신용카드 발급 월
							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도


							=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

In [106]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from kaggler.model import AutoLGB
import random

ModuleNotFoundError: No module named 'kaggler'

In [77]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [78]:
train.shape

(26457, 20)

In [79]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [81]:
total = train.isnull().sum().sort_values(ascending=False)
total2 = test.isnull().sum().sort_values(ascending=False)
print(total)

occyp_type       8171
index               0
gender              0
begin_month         0
family_size         0
email               0
phone               0
work_phone          0
FLAG_MOBIL          0
DAYS_EMPLOYED       0
DAYS_BIRTH          0
house_type          0
family_type         0
edu_type            0
income_type         0
income_total        0
child_num           0
reality             0
car                 0
credit              0
dtype: int64


In [82]:
train.drop(['FLAG_MOBIL'], axis=1, inplace=True)
train.fillna('BaekSu', inplace=True) 

test.drop(['FLAG_MOBIL'], axis=1, inplace=True)
test.fillna('BaekSu', inplace=True)


In [83]:
feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for feat in feats:
    train[feat]=-train[feat]  
    test[feat]=-test[feat]    


In [84]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

total_feats = numerical_feats + categorical_feats

Number of Numerical features:  10
Number of Categorical features:  8


## LabelEncoding, Scaling

In [85]:
for feat in categorical_feats:
    encoder = LabelEncoder()
    train[feat] = encoder.fit_transform(train[feat])
    test[feat] = encoder.transform(test[feat])

In [86]:
num_cols = ['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'work_phone', 'phone', 'email', 'family_size', 'begin_month']

In [87]:
for col in ['child_num', 'family_size']:
    train[col] = np.log(1 + train[col])
    test[col] = np.log(1 + test[col])
    
train['DAYS_BIRTH'] = np.log(1 + train['DAYS_BIRTH'])
test['DAYS_BIRTH'] = np.log(1 + test['DAYS_BIRTH'])

scaler = StandardScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

In [88]:
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,-0.633006,0.149136,0,1,1,2,-0.376794,0.463930,-0.538417,-0.645705,-0.316937,1,-0.092682,-1.215231,1.0
1,1,0,0,1,1.058875,0.590848,0,4,0,1,-1.113878,0.440878,-0.538417,-0.645705,3.155199,9,0.955994,-1.275620,1.0
2,2,1,1,1,-0.633006,2.578550,4,1,1,1,0.792444,0.461929,-0.538417,1.548696,-0.316937,11,-0.092682,-0.249003,2.0
3,3,0,0,1,-0.633006,0.149136,0,4,1,1,-0.074220,0.444893,-0.538417,1.548696,-0.316937,15,-0.092682,0.656836,0.0
4,4,0,1,1,-0.633006,-0.292575,2,1,1,1,-0.086701,0.444988,-0.538417,-0.645705,-0.316937,11,-0.092682,-0.007446,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,0,0,0,2.048562,0.369992,2,4,1,1,-0.894143,0.444108,-0.538417,-0.645705,-0.316937,4,1.769410,-1.456788,1.0
26453,26453,0,0,1,1.058875,-0.071719,4,1,2,1,-0.024954,0.447679,-0.538417,-0.645705,-0.316937,1,-0.092682,1.260729,2.0
26454,26454,0,1,0,-0.633006,1.032559,4,4,0,5,-1.560287,0.444333,-0.538417,-0.645705,-0.316937,4,-0.092682,-0.067835,2.0
26455,26455,1,0,1,-0.633006,-0.160062,4,2,3,1,-1.537326,0.430454,-0.538417,-0.645705,-0.316937,9,-1.570708,1.985400,2.0


### 중복이슈

In [ ]:
train[df_train[train.columns.drop("index")].duplicated(keep = False)].sort_values(["DAYS_BIRTH","income_total"])

In [ ]:
df1 = train[train.drop(["index","begin_month"], axis = 1).duplicated(keep = False)].sort_values(["DAYS_BIRTH","income_total"])

In [ ]:
df1.head(50)

In [ ]:
train.drop(["index","begin_month"], axis = 1).duplicated().sum()

### EDA

In [ ]:
train.groupby(['family_size', 'credit'])['credit'].count()
plt.subplots(figsize=(10,10))
sns.countplot('family_size',hue='credit',data=df_train)
plt.show()

In [ ]:
sns.countplot(y = 'occyp_type', data = train[train['credit']==0.0], order = df_train['occyp_type'].value_counts().index)

In [ ]:
sns.countplot(y = 'occyp_type', data = train[train['credit']==1.0], order = df_train['occyp_type'].value_counts().index)

In [ ]:
sns.countplot(y = 'occyp_type', data = train[train['credit']==2.0], order = df_train['occyp_type'].value_counts().index)

In [ ]:
numerical_feats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

columns = ['income_total','DAYS_BIRTH','DAYS_EMPLOYED','family_size','begin_month']
plt.figure(figsize = (20,15))
for i in range(5):
    plt.subplot(3,2,i+1)
    sns.distplot(train[columns[i]])
plt.show()

### 직업은 없지만 일을 하고 있는 사람이 있다

In [ ]:
train[df_train['DAYS_EMPLOYED'] == 365243]

In [ ]:
train[df_train['occyp_type'] == 'BaekSu']

In [ ]:
# enc = OneHotEncoder()
# enc.fit(df_train.loc[:,categorical_feats])

# df_train_onehot = pd.DataFrame(enc.transform(df_train.loc[:,categorical_feats]).toarray(), 
#              columns=enc.get_feature_names(categorical_feats))
# df_train.drop(categorical_feats, axis=1, inplace=True)
# df_train = pd.concat([df_train, df_train_onehot], axis=1)

# df_train_onehot = pd.DataFrame(enc.transform(df_test.loc[:,categorical_feats]).toarray(), 
#              columns=enc.get_feature_names(categorical_feats))
# df_test.drop(categorical_feats, axis=1, inplace=True)
# df_test = pd.concat([df_test, df_train_onehot], axis=1)

### StratifiedKFold

In [92]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skfold.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

### LGBM

In [93]:
random.seed(42)
lgbm_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgbm = LGBMClassifier(n_estimators=1000)
    lgbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30, verbose=100)
    lgbm_models[fold]=lgbm


====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.64243	valid_1's multi_logloss: 0.757264
[200]	training's multi_logloss: 0.548544	valid_1's multi_logloss: 0.745397
[300]	training's multi_logloss: 0.474781	valid_1's multi_logloss: 0.742925
Early stopping, best iteration is:
[350]	training's multi_logloss: 0.443193	valid_1's multi_logloss: 0.741477
====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.641673	valid_1's multi_logloss: 0.774498
[200]	training's multi_logloss: 0.546452	valid_1's multi_logloss: 0.766442
Early stopping, best iteration is:
[209]	training's multi_logloss: 0.538258	valid_1's multi_logloss: 0.765498
====================================3============================================
Training until validation scores don't

### XGBoost

In [ ]:
random.seed(42)
xgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    
    
    xgb = XGBClassifier(max_depth=5, n_estimators=250, min_child_weight=8, 
                    subsample=0.8, learning_rate =0.02, seed=42)
    xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='logloss')
    xgb_models[fold]=xgb
    



## Modeling

In [89]:
n_est = 1000
seed = 42
n_fold = 5
n_class = 3

rf_params = {
    'max_depth': 15,
    'min_samples_leaf': 8,
    'random_state': seed
}

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': seed,
    'learning_rate': 0.01,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 63,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'num_class': n_class
}

xgb_params = {
    'metric': 'mlogloss',
    'objective': 'multi:softprob',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.01,
    'random_state': seed,
    'num_class': n_class,
    'max_depth': 6,
    'n_estimators': n_est,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
}

In [95]:
models = {'RF': RandomForestClassifier(**rf_params), 
               'LGBM': LGBMClassifier(**lgb_params),
               'XGB': XGBClassifier()}

In [96]:
X = train[total_feats]
y = train['credit']
X_test = test[total_feats]

In [100]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

pred_dic = {}
pred_test_dic = {}
        
for name in models:
    print(f'Training {name}:')
    pred = np.zeros((X.shape[0], 3), dtype=float)
    pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
    for fold in range(5):
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
        models[name].fit(X_train, y_train)
        pred[valid_idx] = models[name].predict_proba(X_valid)
        pred_test += models[name].predict_proba(X_test) / 5
        
    pred_dic[name] = pred
    pred_test_dic[name] = pred_test
    print(f'\tCV Log Loss: {log_loss(y, pred):.6f}')

Training RF:
	CV Log Loss: 0.773210
Training LGBM:
	CV Log Loss: 0.748837
Training XGB:
[22:55:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:55:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:55:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to resto

### Stacking - Failed
ref) https://www.dacon.io/competitions/official/235713/codeshare/2594?page=1&dtype=recent

In [103]:
X = pd.DataFrame(np.hstack([x for _, x in pred_dic.items()]))
X_test = pd.DataFrame(np.hstack([x for _, x in pred_test_dic.items()]))

pred = np.zeros((X.shape[0], 3), dtype=float)
pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
for fold in range(5):
    if fold == 0:
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
        
        model = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': n_class}, 
        feature_selection=False, n_est=10000)
        model.tune(X_train, y_train)
        n_best = model.n_best
        features = model.features
        params = model.params
        
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')      
        print(params)
        model.fit(X_train, y_train)
    else:
        train_data = lgb.Dataset(X[features].iloc[train_idx], label=y[train_idx])
        model = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    pred[valid_idx] = models[name].predict_proba(X_valid)
    pred_test += models[name].predict_proba(X_test) / 5
    

NameError: name 'AutoLGB' is not defined